<a href="https://colab.research.google.com/github/haizzzi/simple-nmt-simplied/blob/main/NMT_Exe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

제일먼저 구글드라이버에 nmt-fold에 code1과 data1이 설치되도록 자신의 구글드라이버에 제공되는 zip 파일을 압축 해제해서 upload 하세요. 그리고 좌측 mount 버튼을 누르세요. 그러면 구글 드라이브 마운트 위치 "/content/drive/MyDrive/"에 마운트가 되어서 /content/drive/MyDrive/nmt_fold/data1/ 형태로 접근이 됩니다.

In [ ]:
from google.colab import drive

# 이미 마운트돼 있으면 다시 연결하고 싶을 때 force_remount=True로
drive.mount('/content/drive')

!pip install torch
!pip install torch_optimizer
!pip install pytorch-ignite

In [ ]:
import torch
print(torch.__version__)


2.6.0+cu124


seq2seq 학습 시키는 코드
10epochs 학습시키면 ppl이 5정도로 내려감.
원래 batch_size가 160이었으나, A100 40GB에 맞춰 960으로 바꿔서 학습 시킴

In [ ]:
!python /content/drive/MyDrive/nmt_fold/simple-nmt/train.py --train /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.bpe --valid /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.bpe --lang enko --gpu_id 0 --batch_size 160 --n_epochs 30 --max_length 64 --dropout .2 --word_vec_size 512 --hidden_size 768 --n_layers 4 --max_grad_norm 1e+8 --iteration_per_update 2 --lr 1e-3 --lr_step 0 --use_adam --model_fn /content/drive/MyDrive/nmt_fold/data1/models1.seq2seq.pth --rl_n_epochs 0

trasformer학습 시키는 코드
10epochs 학습 시켜도 ppl이 17정도로 만족스럽지 않아서 10epochs 더 학습 시킴. batch_size가 원래 128이었으나, A100 40GB에 맞춰서 768으로 바꿔서 학습 시킴

In [ ]:
!python /content/drive/MyDrive/nmt_fold/simple-nmt/train.py --train /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.bpe --valid /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.bpe --lang enko --gpu_id 0 --batch_size 768 --n_epochs 10 --max_length 64 --dropout .2 --hidden_size 768 --n_layers 4 --max_grad_norm 1e+8 --iteration_per_update 32 --lr 1e-3 --lr_step 0 --use_adam --rl_n_epochs 0 --use_transformer --model_fn /content/drive/MyDrive/nmt_fold/data1/models1.transformer.pth

연속해서 학습하는 코드. 10epochs 학습 시킨 모델을 시작으로 20epochs 까지 학습한다. 10epochs에 A100으로 1시반 30분 정도라서 절반씩 나눠서 학습 했다. 20epochs 학습 시키면 ppl이 5.01

In [ ]:
!python /content/drive/MyDrive/nmt_fold/simple-nmt/continue_train.py --load_fn /content/drive/MyDrive/nmt_fold/data1/models1.transformer10.pth --model_fn /content/drive/MyDrive/nmt_fold/data1/models1.transformer.pth --init_epoch 11 --n_epochs 20 --use_transformer

seq2seq10 테스트

In [16]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.bpe.en | python /content/drive/MyDrive/nmt_fold/simple-nmt/translate.py --model_fn /content/drive/MyDrive/nmt_fold/data1/models1.seq2seq10.pth --gpu_id -1 --batch_size 2 --beam_size 1 | python /content/drive/MyDrive/nmt_fold/code1/detokenizer.py
print("==============")
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko
print("==============")
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.en

개방된 공기공간 내 인근 건물이 조성한 자체적인 집집이지만 흡연자들이 쏟아내는 무분별한 담배연기 연기가 시민들이 몰려들어 시민들이 몰려들고 있는 공원 밖의 보행자길으로 확대됐다.
오리지널의 생리대 라인과는 차이가 있나요?
"이 가을이 다가오기 전에도 창덕궁과 하정당을 찾아, 이곳에 반사된 시대의 아픔을 느끼는 것도 좋다."
"1차 1차는 전혀 반응하지 않고, 약물의 초기 효과를 보여주는 환자의 1분의 1가량이 약한 강도가 되어 약이 안 나온다."
그는 “경제성을 배제하고 배제할 수 있는 핵심은 기업 소유구조”라며 “기업 소유구조는 기업 소유구조”라며 향후 사업모델이 ‘참여기업’이 될 것이라고 전망했다.
이곳은 공개공지내 인근 건물에서 자체 조성한 공원이지만 흡연자들이 뿜어대는 무분별한 담배연기가 공원외부 보행로까지 확대되어 시민들의 눈살을 찌뿌리게 한 곳이다.
미디헴팬티는 기존에 있는 속옷과 다른 것입니까?
이 가을이 다 가기 전에 창덕궁과 희정당을 찾아 그 속에 담긴 시대적인 아픔을 한번 느껴보는 것도 좋을 것 같다.
"약 3분의 1은 전혀 반응을 보이지 않으며, 투약 초기 효과를 보이던 환자들도 약 3분의 1은 내성이 생기는지 점차 약발이 듣지 않게 된다."
"그는 “경제적 포용과 배제를 가르는 관건은 기업의 소유구조”라며, 소유권이 소수에게 집중된 기존 경제모델이 힘을 잃어가는 반면 미래 비즈니스 모델은 ‘참여적 기업’이 될 것이라고 전망했다."
"It is a self-created park created by nearby buildings within the open air space, but reckless cigarette smoke spewed by smokers has expanded to pedestrian paths outside the park, which is frowned upon by the citizens."
Is the midi hem panty different from the original underwear line?
"Be

transformer20 테스트

In [15]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.bpe.en | python /content/drive/MyDrive/nmt_fold/simple-nmt/translate.py --model_fn /content/drive/MyDrive/nmt_fold/data1/models1.transformer20.pth --gpu_id -1 --batch_size 2 --beam_size 1 | python /content/drive/MyDrive/nmt_fold/code1/detokenizer.py
print("==============")
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko
print("==============")
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.en

"공개된 공중내 건물 내에 인근 건물들이 만든 셀프공원이지만, 시민들이 몰던 공원 밖 보행길까지 확대해 시민들이 몰리고 있는 것이다."
그가 원하는 옷라인과 다른 중간품은 있나요?
가을이 되기 전에도 창덕궁과 하숙당을 찾아 현장에 반영된 시대의 고통을 느껴보는 것도 좋다.
"약 1,3명의 환자가 모두 반응하지 않고, 약을 먹는 초기 효과를 보인 환자 중 3분의 1 정도가 적어 약이 킥이 나지 않는다."
그는 “경제적으로는 기업의 소유권이자 배제되는 핵심이 기업 소유구조”라며 “미래 비즈니스 모델이 도약할 것”이라고 전망했다.
이곳은 공개공지내 인근 건물에서 자체 조성한 공원이지만 흡연자들이 뿜어대는 무분별한 담배연기가 공원외부 보행로까지 확대되어 시민들의 눈살을 찌뿌리게 한 곳이다.
미디헴팬티는 기존에 있는 속옷과 다른 것입니까?
이 가을이 다 가기 전에 창덕궁과 희정당을 찾아 그 속에 담긴 시대적인 아픔을 한번 느껴보는 것도 좋을 것 같다.
"약 3분의 1은 전혀 반응을 보이지 않으며, 투약 초기 효과를 보이던 환자들도 약 3분의 1은 내성이 생기는지 점차 약발이 듣지 않게 된다."
"그는 “경제적 포용과 배제를 가르는 관건은 기업의 소유구조”라며, 소유권이 소수에게 집중된 기존 경제모델이 힘을 잃어가는 반면 미래 비즈니스 모델은 ‘참여적 기업’이 될 것이라고 전망했다."
"It is a self-created park created by nearby buildings within the open air space, but reckless cigarette smoke spewed by smokers has expanded to pedestrian paths outside the park, which is frowned upon by the citizens."
Is the midi hem panty different from the original underwear line?
"Before this fall is out, it should also be goo

transformer20 테스트 다른 위치

In [14]:
!head -n 5000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.bpe.en | tail -n 5 | python /content/drive/MyDrive/nmt_fold/simple-nmt/translate.py --model_fn /content/drive/MyDrive/nmt_fold/data1/models1.transformer20.pth --gpu_id -1 --batch_size 2 --beam_size 1 | python /content/drive/MyDrive/nmt_fold/code1/detokenizer.py
print("==============")
!head -n 5000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko | tail -n 5
print("==============")
!head -n 5000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.en | tail -n 5


제14조에 따른 “서울특별시 관악구 사회단체 보조금 지원 조례”를 “서울특별시 관악구 지방보조금”으로 한다.
국립기상과학원이 수월봉에 있다.
게다가 시장은 너무 조용해서 오후에 사람이 없어.
당신은 삼성에서 일하고 있나요?
이 제품은 중국에서 생산되므로 박스에 문구가 A로 변경되어야 합니다.
제14조 중 “서울특별시 관악구 사회단체 보조금 지원 조례”를 “서울 특별시 관악구 지방보조금 관리 조례”로 한다.
수월봉에 국립기상관측소가 있어요.
게다가 오후가 되면 사람들을 볼 수 없을 정도로 시장이 한산하다.
당신은 옌빈에 있는 삼성에서 일합니까?
이 제품은 중국에서 생산하는 제품이므로 박스의 문구가 a로 변경되어야 해요.
"In Article 14, the ""Gwanak-gu Seoul Metropolitan Government Ordinance on the Support of Subsidies for Social Organizations"" shall be the ""Gwanak-gu Seoul Metropolitan Government Ordinance on Local Subsidies."""
National Institute of Meteorological Research is at Suwolbong.
"Moreover, the market is so quiet that there are no people in the afternoon."
Do you work for Samsung in Yen Binh?
"This product is produced in China, so the phrase on the box should be changed to A."
